In [1]:
import pymysql
from datetime import datetime as dt
import pandas as pd

In [70]:
class PortfolioTracker():
    def __init__(self):
        self.conn = pymysql.connect(
            host='localhost',
            user='root', 
            password = "password",
            port=6603,
            db='PortfolioTracker',
            )

    def find_asset(self, idcustom):
        c = self.conn.cursor()
        sql = 'Select idasset from assets where idcustom=%s'
        c.execute(sql, (idcustom,) )
        idasset = c.fetchone()
        return idasset

    def find_portfolio(self, idcustom):
        c = self.conn.cursor()
        sql = 'Select idportfolio from portfolios where idcustom=%s'
        c.execute(sql, (idcustom,) )
        idportfolio = c.fetchone()
        return idportfolio

    def create_asset(self, idcustom, name):
        c = self.conn.cursor()
        sql = 'INSERT INTO assets (idcustom, name) VALUES (%s, %s)'
        c.execute(sql, (idcustom, name,))
        self.conn.commit()
        return self.find_asset(idcustom)

    def create_portfolio(self, idcustom, name):
        c = self.conn.cursor()
        sql = 'INSERT INTO portfolios (idcustom, name) VALUES (%s, %s)'
        c.execute(sql, (idcustom, name,))
        self.conn.commit()
        return self.find_portfolio(idcustom)
    
    def load_trade(self, idportfolio, idasset, date, qty, price):
        c = self.conn.cursor()
        sql = 'INSERT INTO trades (idportfolio, idasset, date, qty, price) VALUES (%s, %s, %s, %s, %s)'
        c.execute(sql, (idportfolio, idasset, dt.strptime(date,'%d/%m/%Y'), qty, price))
        self.conn.commit()
        return 0
    
    def load_price(self, idasset, date, price):
        c = self.conn.cursor()
        sql = 'INSERT INTO prices (idasset, date, price) VALUES (%s, %s, %s)'
        c.execute(sql, (idasset, dt.strptime(date,'%d/%m/%Y'), price))
        self.conn.commit()
        return 0
    
    def find_price(self, date, idasset):
        c = self.conn.cursor()
        sql = 'Select price from prices where idasset=%s and date=%s'
        c.execute(sql, (idasset, dt.strptime(date,'%d/%m/%Y')) )
        price = c.fetchone()
        if price is None:
            print('Price not found for idasset {0}'.format(idasset))
            price = 0
        return price[0]
    
    def find_prices(self, date, idassets):
        prices = []
        for idasset in idassets:
            prices.append(self.find_price(date, idasset))
        return prices
    
    def load_price_from_file(self, path):
        df = pd.read_csv(path)
        for val in df.values:
            aname = self.find_asset(val[0])
            if aname is None:
                aname = self.create_asset(val[1])
            self.load_price(aname, *val[1:])
    
    def load_trade_from_file(self, path):
        df = pd.read_csv(path)
        for val in df.values:
            #find asset/portfolio
            pname = self.find_portfolio(val[0])
            aname = self.find_asset(val[1])
            #create if not found
            if pname is None:
                pname = self.create_portfolio(val[0], val[0])
            if aname is None:
                aname = self.create_asset(val[1], val[1])
            #load trade
            self.load_trade(pname, aname, *val[2:])
    
    def get_valuation_df(self, date, portfolio_name):
        c = self.conn.cursor()
        idp = myp.find_portfolio(portfolio_name)
        sql = 'Select * from trades where idportfolio=%s and date<=%s'
        c.execute(sql,(idp, dt.strptime(date,'%d/%m/%Y'),))
        #create trade dataframe
        rdf = pd.DataFrame(c.fetchall())
        rdf.columns = [i[0] for i in c.description]
        
        #aggeragate for valuation
        aggdf = rdf.groupby(['idasset']).sum()
        
        #get prices as of requested date
        curr_prices = self.find_prices(date, aggdf.index.values)
        aggdf['price'] = curr_prices
        aggdf['value'] = aggdf['price']*aggdf['qty']
        
        return aggdf
#TODO: create price loader, create functions to request trades and valuations for date/portfolio 


In [68]:
myp = PortfolioTracker()
c = myp.conn.cursor()
idp = myp.find_portfolio('PortA')
date = '15/05/2022'
sql = 'Select * from trades where idportfolio=%s and date<=%s'
c.execute(sql,(idp, dt.strptime(date,'%d/%m/%Y'),))
rdf = pd.DataFrame(c.fetchall())
rdf.columns = [i[0] for i in c.description]
aggdf = rdf.groupby(['idasset']).sum()
curr_prices = myp.find_prices(date, aggdf.index.values)
aggdf['price'] = curr_prices
aggdf['value'] = aggdf['price']*aggdf['qty']
aggdf
#aggdf.index.values
# for x in aggdf.index.values:
#     print(x)

,idportfolio,qty,price,value
idasset,,,,
11,36,200,10.00,2000.00
12,36,400,10.00,4000.00
13,36,600,10.00,6000.00


In [71]:
myp = PortfolioTracker()
myp.get_valuation_df('15/05/2022', 'PortA')

,idportfolio,qty,price,value
idasset,,,,
11,36,200,10.00,2000.00
12,36,400,10.00,4000.00
13,36,600,10.00,6000.00


datetime.datetime(2022, 4, 16, 0, 0)